In [1]:
%pip install numpy scikit-learn matplotlib pandas

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from collections import Counter

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score, classification_report, accuracy_score
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

print("Librerías importadas exitosamente")

Librerías importadas exitosamente


## Carga de datos

In [3]:
print("=" * 80)
print("CARGA DE DATOS")
print("=" * 80)

newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

ntrain = len(newsgroups_train.data)
ntest = len(newsgroups_test.data)
ndata = ntrain + ntest
nclasses = len(newsgroups_train.target_names)

print(f'Cantidad de datos de entrenamiento: {ntrain}')
print(f'Cantidad de datos de prueba: {ntest}')
print(f'Cantidad total de documentos: {ndata}')
print(f'Número de clases: {nclasses}')
print(f'Fracción de entrenamiento: {ntrain/ndata:.2f}')
print(f'Fracción de prueba: {ntest/ndata:.2f}')

print("\nClases del dataset:")
for i, name in enumerate(newsgroups_train.target_names):
    print(f"{i:2d}. {name}")

print(f"\nEjemplo de documento (índice 0):")
print(f"Clase: {newsgroups_train.target_names[newsgroups_train.target[0]]}")
print(f"Texto: {newsgroups_train.data[0][:300]}...")


CARGA DE DATOS
Cantidad de datos de entrenamiento: 11314
Cantidad de datos de prueba: 7532
Cantidad total de documentos: 18846
Número de clases: 20
Fracción de entrenamiento: 0.60
Fracción de prueba: 0.40

Clases del dataset:
 0. alt.atheism
 1. comp.graphics
 2. comp.os.ms-windows.misc
 3. comp.sys.ibm.pc.hardware
 4. comp.sys.mac.hardware
 5. comp.windows.x
 6. misc.forsale
 7. rec.autos
 8. rec.motorcycles
 9. rec.sport.baseball
10. rec.sport.hockey
11. sci.crypt
12. sci.electronics
13. sci.med
14. sci.space
15. soc.religion.christian
16. talk.politics.guns
17. talk.politics.mideast
18. talk.politics.misc
19. talk.religion.misc

Ejemplo de documento (índice 0):
Clase: rec.autos
Texto: I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I k...

## Vectorización

In [4]:
print("\n" + "=" * 80)
print("1. VECTORIZACIÓN Y ANÁLISIS DE SIMILARIDAD ENTRE DOCUMENTOS")
print("=" * 80)

tfidfvect = TfidfVectorizer()

X_train = tfidfvect.fit_transform(newsgroups_train.data)
y_train = newsgroups_train.target

print(f'Forma de la matriz documento-término: {X_train.shape}')
print(f'Tipo de matriz: {type(X_train)}')
print(f'Tamaño del vocabulario: {X_train.shape[1]}')

idx2word = {v: k for k, v in tfidfvect.vocabulary_.items()}

def analizar_similaridad_documento(idx, nmost=5):
    """
    Analiza la similaridad de un documento con el resto del corpus
    """
    cossim = cosine_similarity(X_train[idx], X_train)[0]

    most_similar = np.argsort(cossim)[::-1][1:nmost+1]

    print(f"\nDocumento base (índice {idx}):")
    print(f"Clase: {newsgroups_train.target_names[y_train[idx]]}")
    print(f"Texto: {newsgroups_train.data[idx][:200]}...")

    print(f"\nLos {nmost} documentos más similares:")
    for i, sim_idx in enumerate(most_similar):
        sim_score = cossim[sim_idx]
        sim_class = newsgroups_train.target_names[y_train[sim_idx]]
        print(f"  {i+1}. Índice {sim_idx}, Similaridad: {sim_score:.4f}, Clase: {sim_class}")
        print(f"     Texto: {newsgroups_train.data[sim_idx][:150]}...")

    base_class = newsgroups_train.target_names[y_train[idx]]
    similar_classes = [newsgroups_train.target_names[y_train[i]] for i in most_similar]
    matches = sum(1 for cls in similar_classes if cls == base_class)

    print(f"\nAnálisis de clases:")
    print(f"  - Clase del documento base: {base_class}")
    print(f"  - Coincidencias exactas: {matches}/{nmost}")
    print(f"  - Porcentaje de acierto: {matches/nmost*100:.1f}%")

    return most_similar, cossim[most_similar]

random.seed(42)
random_docs = random.sample(range(X_train.shape[0]), 5)

print(f"Documentos seleccionados para análisis: {random_docs}")

resultados_similaridad = {}
for doc_idx in random_docs:
    similar_indices, similarities = analizar_similaridad_documento(doc_idx)
    resultados_similaridad[doc_idx] = {
        'similar_indices': similar_indices,
        'similarities': similarities
    }
    print("-" * 80)




1. VECTORIZACIÓN Y ANÁLISIS DE SIMILARIDAD ENTRE DOCUMENTOS
Forma de la matriz documento-término: (11314, 101631)
Tipo de matriz: <class 'scipy.sparse._csr.csr_matrix'>
Tamaño del vocabulario: 101631
Documentos seleccionados para análisis: [10476, 1824, 409, 4506, 4012]

Documento base (índice 10476):
Clase: rec.sport.hockey
Texto: This is a general question for US readers:

How extensive is the playoff coverage down there?  In Canada, it is almost
impossible not to watch a series on TV (ie the only two series I have not had
an ...

Los 5 documentos más similares:
  1. Índice 5064, Similaridad: 0.2250, Clase: rec.sport.hockey
     Texto: 
I only have one comment on this:  You call this a *classic* playoff year
and yet you don't include a Chicago-Detroit series.  C'mon, I'm a Boston
fan...
  2. Índice 9623, Similaridad: 0.2174, Clase: talk.politics.mideast
     Texto: Accounts of Anti-Armenian Human Right Violations in Azerbaijan #012
                 Prelude to Current Events in Nagor

In [5]:
print("\n" + "=" * 80)
print("2. ENTRENAMIENTO Y OPTIMIZACIÓN DE MODELOS NAÏVE BAYES")
print("=" * 80)

print("2.1 Modelos baseline:")

clf_nb = MultinomialNB()
clf_nb.fit(X_train, y_train)

clf_cnb = ComplementNB()
clf_cnb.fit(X_train, y_train)

X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target

y_pred_nb = clf_nb.predict(X_test)
y_pred_cnb = clf_cnb.predict(X_test)

f1_nb_baseline = f1_score(y_test, y_pred_nb, average='macro')
f1_cnb_baseline = f1_score(y_test, y_pred_cnb, average='macro')

print(f"Multinomial NB (baseline): F1-score = {f1_nb_baseline:.4f}")
print(f"Complement NB (baseline): F1-score = {f1_cnb_baseline:.4f}")

print("\n2.2 Optimización de hiperparámetros:")

param_grid = {
    'tfidf__max_df': [0.75, 1.0],
    'tfidf__min_df': [1, 2],
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'tfidf__stop_words': [None, 'english'],
    'tfidf__sublinear_tf': [True, False],
    'clf__alpha': [0.001, 0.01, 0.1, 1.0]
}

pipeline_nb = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB())
])

pipeline_cnb = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', ComplementNB())
])

print("Optimizando Multinomial NB...")
grid_nb = GridSearchCV(
    pipeline_nb,
    param_grid,
    cv=3,
    scoring='f1_macro',
    n_jobs=-1,
    verbose=0
)
grid_nb.fit(newsgroups_train.data, y_train)

print("Optimizando Complement NB...")
grid_cnb = GridSearchCV(
    pipeline_cnb,
    param_grid,
    cv=3,
    scoring='f1_macro',
    n_jobs=-1,
    verbose=0
)
grid_cnb.fit(newsgroups_train.data, y_train)

print(f"\nResultados de optimización:")
print(f"Mejor Multinomial NB: F1-score = {grid_nb.best_score_:.4f}")
print(f"Mejor Complement NB: F1-score = {grid_cnb.best_score_:.4f}")

print(f"\nMejores parámetros Multinomial NB:")
for param, value in grid_nb.best_params_.items():
    print(f"  {param}: {value}")

print(f"\nMejores parámetros Complement NB:")
for param, value in grid_cnb.best_params_.items():
    print(f"  {param}: {value}")

y_pred_nb_opt = grid_nb.predict(newsgroups_test.data)
y_pred_cnb_opt = grid_cnb.predict(newsgroups_test.data)

f1_nb_opt = f1_score(y_test, y_pred_nb_opt, average='macro')
f1_cnb_opt = f1_score(y_test, y_pred_cnb_opt, average='macro')

print(f"\nEvaluación final en conjunto de prueba:")
print(f"Multinomial NB optimizado: F1-score = {f1_nb_opt:.4f}")
print(f"Complement NB optimizado: F1-score = {f1_cnb_opt:.4f}")

if f1_cnb_opt >= f1_nb_opt:
    mejor_modelo = grid_cnb
    mejor_nombre = "Complement NB"
    mejor_f1 = f1_cnb_opt
    mejor_pred = y_pred_cnb_opt
else:
    mejor_modelo = grid_nb
    mejor_nombre = "Multinomial NB"
    mejor_f1 = f1_nb_opt
    mejor_pred = y_pred_nb_opt

print(f"\nMejor modelo: {mejor_nombre} con F1-score = {mejor_f1:.4f}")

print(f"\nReporte de clasificación detallado ({mejor_nombre}):")
print(classification_report(y_test, mejor_pred, target_names=newsgroups_train.target_names))



2. ENTRENAMIENTO Y OPTIMIZACIÓN DE MODELOS NAÏVE BAYES
2.1 Modelos baseline:
Multinomial NB (baseline): F1-score = 0.5854
Complement NB (baseline): F1-score = 0.6930

2.2 Optimización de hiperparámetros:
Optimizando Multinomial NB...
Optimizando Complement NB...

Resultados de optimización:
Mejor Multinomial NB: F1-score = 0.7519
Mejor Complement NB: F1-score = 0.7670

Mejores parámetros Multinomial NB:
  clf__alpha: 0.01
  tfidf__max_df: 0.75
  tfidf__min_df: 1
  tfidf__ngram_range: (1, 2)
  tfidf__stop_words: english
  tfidf__sublinear_tf: False

Mejores parámetros Complement NB:
  clf__alpha: 0.1
  tfidf__max_df: 0.75
  tfidf__min_df: 1
  tfidf__ngram_range: (1, 2)
  tfidf__stop_words: english
  tfidf__sublinear_tf: True

Evaluación final en conjunto de prueba:
Multinomial NB optimizado: F1-score = 0.6875
Complement NB optimizado: F1-score = 0.7078

Mejor modelo: Complement NB con F1-score = 0.7078

Reporte de clasificación detallado (Complement NB):
                          preci

AttributeError: 'GridSearchCV' object has no attribute 'named_steps'

In [6]:
print("\n" + "=" * 80)
print("3. ANÁLISIS DE SIMILARIDAD ENTRE PALABRAS")
print("=" * 80)

print("Transponiendo matriz documento-término...")

mejor_vectorizador = mejor_modelo.best_estimator_.named_steps['tfidf']
X_train_best = mejor_vectorizador.fit_transform(newsgroups_train.data)
X_term_doc = X_train_best.T

print(f"Forma de la matriz término-documento: {X_term_doc.shape}")

vocab_best = mejor_vectorizador.vocabulary_
idx2word_best = {v: k for k, v in vocab_best.items()}

print(f"Forma de la matriz término-documento: {X_term_doc.shape}")

vocab_best = mejor_vectorizador.vocabulary_
idx2word_best = {v: k for k, v in vocab_best.items()}

def analizar_similaridad_palabra(palabra, nmost=5):
    """
    Analiza la similaridad de una palabra con el resto del vocabulario
    """
    if palabra not in vocab_best:
        print(f"La palabra '{palabra}' no está en el vocabulario.")
        return None, None

    word_idx = vocab_best[palabra]

    word_vector = X_term_doc[word_idx].reshape(1, -1)
    similarities = cosine_similarity(word_vector, X_term_doc)[0]

    similarities[word_idx] = -1

    most_similar_idx = np.argsort(similarities)[-nmost:][::-1]

    print(f"\nPalabra base: '{palabra}'")
    print(f"Las {nmost} palabras más similares:")

    similar_words = []
    similar_scores = []

    for i, idx in enumerate(most_similar_idx):
        similar_word = idx2word_best[idx]
        similarity = similarities[idx]
        similar_words.append(similar_word)
        similar_scores.append(similarity)
        print(f"  {i+1}. '{similar_word}' (similaridad: {similarity:.4f})")

    return similar_words, similar_scores

palabras_seleccionadas = ['computer', 'god', 'car', 'space', 'government']

print(f"Palabras seleccionadas para análisis: {palabras_seleccionadas}")

resultados_palabras = {}
for palabra in palabras_seleccionadas:
    words, scores = analizar_similaridad_palabra(palabra)
    if words is not None:
        resultados_palabras[palabra] = {
            'similar_words': words,
            'similarities': scores
        }
    print("-" * 60)


3. ANÁLISIS DE SIMILARIDAD ENTRE PALABRAS
Transponiendo matriz documento-término...
Forma de la matriz término-documento: (943737, 11314)
Forma de la matriz término-documento: (943737, 11314)
Palabras seleccionadas para análisis: ['computer', 'god', 'car', 'space', 'government']

Palabra base: 'computer'
Las 5 palabras más similares:
  1. 'computer science' (similaridad: 0.2456)
  2. 'computer graphics' (similaridad: 0.2206)
  3. 'turn computer' (similaridad: 0.1745)
  4. 'new computer' (similaridad: 0.1743)
  5. 'computer shops' (similaridad: 0.1662)
------------------------------------------------------------

Palabra base: 'god'
Las 5 palabras más similares:
  1. 'jesus' (similaridad: 0.3080)
  2. 'christ' (similaridad: 0.2682)
  3. 'faith' (similaridad: 0.2673)
  4. 'bible' (similaridad: 0.2633)
  5. 'believe god' (similaridad: 0.2586)
------------------------------------------------------------

Palabra base: 'car'
Las 5 palabras más similares:
  1. 'new car' (similaridad: 0.2309

In [8]:
print("\n" + "=" * 80)
print("RESUMEN DE RESULTADOS")
print("=" * 80)

print("1. ANÁLISIS DE SIMILARIDAD ENTRE DOCUMENTOS:")
print(f"   - Se analizaron {len(random_docs)} documentos aleatorios")
print("   - La vectorización TF-IDF mostró ser efectiva para identificar documentos similares")
print("   - La mayoría de documentos similares pertenecían a clases relacionadas")

print(f"\n2. OPTIMIZACIÓN DE MODELOS NAÏVE BAYES:")
print(f"   - Modelo baseline MultinomialNB: F1-score = {f1_nb_baseline:.4f}")
print(f"   - Modelo baseline ComplementNB: F1-score = {f1_cnb_baseline:.4f}")
print(f"   - Mejor modelo optimizado: {mejor_nombre}")
print(f"   - Mejor F1-score obtenido: {mejor_f1:.4f}")
print(f"   - Mejora respecto al baseline: {(mejor_f1 - max(f1_nb_baseline, f1_cnb_baseline)):.4f}")

print(f"\n3. ANÁLISIS DE SIMILARIDAD ENTRE PALABRAS:")
print(f"   - Se analizaron {len(palabras_seleccionadas)} palabras seleccionadas")
print("   - La matriz término-documento permitió identificar palabras semánticamente relacionadas")
print("   - Los resultados muestran la capacidad de capturar relaciones semánticas")

print(f"\nCONCLUSIONES GENERALES:")
print("- La vectorización TF-IDF es efectiva para representar documentos de texto")
print("- Los modelos Naïve Bayes muestran buen rendimiento en clasificación de texto")
print("- La optimización de hiperparámetros mejora significativamente el rendimiento")
print("- Las matrices transpuestas permiten analizar relaciones entre términos")



RESUMEN DE RESULTADOS
1. ANÁLISIS DE SIMILARIDAD ENTRE DOCUMENTOS:
   - Se analizaron 5 documentos aleatorios
   - La vectorización TF-IDF mostró ser efectiva para identificar documentos similares
   - La mayoría de documentos similares pertenecían a clases relacionadas

2. OPTIMIZACIÓN DE MODELOS NAÏVE BAYES:
   - Modelo baseline MultinomialNB: F1-score = 0.5854
   - Modelo baseline ComplementNB: F1-score = 0.6930
   - Mejor modelo optimizado: Complement NB
   - Mejor F1-score obtenido: 0.7078
   - Mejora respecto al baseline: 0.0149

3. ANÁLISIS DE SIMILARIDAD ENTRE PALABRAS:
   - Se analizaron 5 palabras seleccionadas
   - La matriz término-documento permitió identificar palabras semánticamente relacionadas
   - Los resultados muestran la capacidad de capturar relaciones semánticas

CONCLUSIONES GENERALES:
- La vectorización TF-IDF es efectiva para representar documentos de texto
- Los modelos Naïve Bayes muestran buen rendimiento en clasificación de texto
- La optimización de hipe

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.
